# Chat History Yönetimli RAG Chatbot Uygulaması

Yüklediğiniz PDF/TXT/DOCX belgelerden bilgi retrieve eden, OpenAI destekli RAG Chatbot örneği. 
Ayrıca, kullanıcı sohbet geçmişini kaydeder, arayabilir, temizleyebilir ve tekrar yükleyebilirsiniz.

### Kullanacağımız teknoloji stack'i:
- OpenAI (chat ve embedding modelleri)
- FAISS (vektör tabanlı arama için)
- LangChain (framework)
- Python (klasik notebook arayüzü) -> daha sonra streamlit
- Chat History yönetimi (sadece context olarak değil, örnekleme/analiz için de)

### Senaryo
- Kullanıcı PDF/DOCX/TXT belgesi yüklüyor.
- Belgelerden embedding'ler çıkarılıyor ve FAISS ile arama index'i oluşturuluyor.
- Kullanıcı, soru soruyor.
- Soru embedding'e dönüştürülüp, vektör arama ile en ilgili context'ler bulunuyor.
- Sonrasında OpenAI Chat API'ya hem kullanıcı geçmişi hem de ilgili doküman parçaları eklenip yanıt oluşturuluyor.
- Chat geçmişi, her turda context olarak prompt'a ekleniyor ve ayrı bir yerde de saklanıyor.

---

### 1. Gerekli Kütüphanelerin Kurulumu

In [ ]:
!pip install openai langchain faiss-cpu tiktoken PyPDF2 python-docx

### 2. Kütüphanelerin İçe Aktarılması

In [ ]:
# General Imports
import os
import uuid
import json
import getpass
import textwrap
import datetime
# LLM Imports
import faiss
import openai
# Langchain Imports
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document as LCDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Documantation Imports
from docx import Document
from PyPDF2 import PdfReader

### 3. OpenAI API Key Tanımlama

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    openai_api_key = getpass.getpass("OpenAI API Key'inizi giriniz: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

### 4. Belge Okuma Fonksiyonları

In [ ]:
def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ''
    for page in reader.pages:
        if page.extract_text():
            text += page.extract_text() + "\n"
    return text

def read_docx(file_path):
    doc = Document(file_path)
    return '\n'.join([para.text for para in doc.paragraphs])

def load_document(file_path):
    if file_path.endswith('.txt'):
        return read_txt(file_path)
    elif file_path.endswith('.pdf'):
        return read_pdf(file_path)
    elif file_path.endswith('.docx'):
        return read_docx(file_path)
    else:
        raise ValueError("Desteklenmeyen dosya formatı")

### 5. Belge Chunk'lama

In [ ]:
def chunk_document(document, chunk_size=500, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_text(document)

### 6. Chat History Manager Sınıfı

In [ ]:
class ChatHistoryManager:
    def __init__(self, user_id, base_path="chat_histories"):
        self.user_id = user_id
        self.base_path = base_path
        self.history = []
        os.makedirs(self.base_path, exist_ok=True)

    def add_message(self, role, message):
        self.history.append({
            "role": role,
            "message": message,
            "timestamp": datetime.datetime.now().isoformat()
        })

    def get_history(self, n_last=None):
        if n_last is None:
            return self.history
        return self.history[-n_last:]

    def save(self):
        file_path = os.path.join(self.base_path, f"{self.user_id}_history.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(self.history, f, ensure_ascii=False, indent=2)

    def load(self):
        file_path = os.path.join(self.base_path, f"{self.user_id}_history.json")
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                self.history = json.load(f)
        else:
            self.history = []

    def clear(self):
        self.history = []
        self.save()

    def search(self, keyword):
        return [msg for msg in self.history if keyword.lower() in msg["message"].lower()]

### 7. Belge Yükleme

**Not:** Burada notebook çalıştırdığınız yerde bir dosya olması lazım.
PDF/TXT/DOCX dosyanızın adını aşağıda belirtin (ör: example.pdf)

In [ ]:
file_path = "Profile.pdf"
document = load_document(file_path)
chunks = chunk_document(document)
print(f"{len(chunks)} adet chunk oluşturuldu.")

# Her chunk için kaynak bilgisini hazırla
sources = [f"Chunk {i+1}: {chunk[:60].replace('\n', ' ')}..." for i, chunk in enumerate(chunks)]

### 8. FAISS Index ve Embedding

In [ ]:
embeddings = OpenAIEmbeddings()
docs_with_metadata = [
    LCDocument(page_content=chunk, metadata={"source": sources[i]})
    for i, chunk in enumerate(chunks)
]
db = FAISS.from_documents(docs_with_metadata, embeddings)

### 9. Kullanıcı Tanımlama ve Chat Geçmişi Yönetimi

In [ ]:
user_id = str(uuid.uuid4())[:8]  # otomatik ve benzersiz id
print(f"Oturumunuz için otomatik atanmış kullanıcı id: {user_id}")
history_manager = ChatHistoryManager(user_id)
history_manager.load()

### 10. Retrieval ve OpenAI API Fonksiyonları

In [ ]:
def get_relevant_docs_with_source(query, db, k=4, threshold=0.45):
    docs_and_scores = db.similarity_search_with_score(query, k=k)
    filtered = [
        (doc.page_content, doc.metadata.get("source", "kaynak yok"))
        for doc, score in docs_and_scores
        # NOT: FAISS skorları genellikle mesafe, daha küçük = daha yakın. 
        # Burada 0.4 gibi düşük threshold daha "sıkı" seçimdir.
        if score < threshold # eşik altında ise gerçekten ilgili chunk kabul et
    ]
    docs, sources = zip(*filtered) if filtered else ([], [])
    return list(docs), list(sources)

def format_context_with_sources(docs, sources):
    if not docs:
        return "(Bu soruyla ilgili kaynak chunk bulunamadı.)"
    return "\n\n".join([f"[{source}]\n{doc}" for source, doc in zip(sources, docs)])

def format_chat_history(history):
    return "\n".join([f"{role}: {msg}" for role, msg in history[-6:]])

def build_chat_prompt_with_sources(user_input, docs, sources, chat_history):
    context = format_context_with_sources(docs, sources)
    history_str = format_chat_history(chat_history)
    prompt = f"""
        Aşağıda kullanıcı sorusuyla ilgili belge parçaları bulunmaktadır. Bunlara dayalı şekilde, net, referanslı ve açıklayıcı bir yanıt ver.
        
        Belge Parçaları:
        {context}
        
        Geçmiş Sohbet:
        {history_str}
        
        Kullanıcı Sorusu:
        {user_input}
    """
    return textwrap.dedent(prompt).strip()

### 11. Chatbot Döngüsü (Chat History Yönetimi ile)

**Komutlar:**
- **history:** Tüm chat geçmişini gösterir
- **clear:** Sohbet geçmişini siler
- **search kelime:** Geçmişte anahtar kelime arar
- **quit veya exit:** Sohbeti bitirir

In [ ]:
# OpenAI istemcisiyle başlat 
client = openai.OpenAI(api_key=openai_api_key)

while True:
    user_input = input("\nSen: ").strip()
    print("\nSen:", user_input)
    user_input_lower = user_input.lower()

    if user_input_lower in {"quit", "exit"}:
        print("Çıkılıyor.")
        break

    if user_input_lower == "history":
        print("\n--- Sohbet Geçmişi ---")
        for msg in history_manager.get_history():
            print(f"{msg['role']} ({msg['timestamp']}): {msg['message']}")
        continue

    if user_input_lower == "clear":
        history_manager.clear()
        print("Sohbet geçmişi temizlendi.")
        continue

    if user_input_lower.startswith("search "):
        keyword = user_input.split(" ", 1)[1]
        results = history_manager.search(keyword)
        print(f"\n--- '{keyword}' için geçmiş mesajlar ---")
        for msg in results:
            print(f"{msg['role']} ({msg['timestamp']}): {msg['message']}")
        continue

    # RAG: En alakalı chunk'ları al
    relevant_docs, sources = get_relevant_docs_with_source(user_input, db)

    # Geçmişi al ve prompt'u oluştur
    history_last = [(msg["role"], msg["message"]) for msg in history_manager.get_history(6)]
    prompt_text = build_chat_prompt_with_sources(user_input, relevant_docs, sources, history_last)
    
    # messages dizisi: system + context + geçmiş + yeni soru
    messages = [
        {
            "role": "system",
            "content": (
                "Sen profesyonel bir yardımcı asistansın. "
                "Kullanıcı sorularını, verilen belge parçaları ve geçmiş konuşmaya göre yanıtla. "
                "Kaynaklardan aldığın bilgiye referans vermeyi unutma."
            )
        },
        {"role": "user", "content": prompt_text}
    ]
    
    # Assistan yanıtı
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=512,
        temperature=0.7,
    )

    assistant_response = response.choices[0].message.content
    print("\nAsistan:", assistant_response)

    print("\nKullanılan kaynak chunk'lar:")
    if sources:
        for s in sources:
            print("-", s)
    else:
        print("- Hiçbir kaynak bulunamadı.")

    # Geçmişe ekle
    history_manager.add_message("user", user_input)
    history_manager.add_message("assistant", assistant_response)
    history_manager.save()

### 13. Sohbet Geçmişini Dışa Aktar

Geçmişi bir data frame olarak dışarı aktaralım ve veri analizi için pandas ile okuyalım.

In [ ]:
import pandas as pd
df = pd.DataFrame(history_manager.get_history())
df.head()